### Group ID: HaKaPr
● Submitted by:
1. Kanishk Chugh (2004215)<br>
2. Prem Swarup (2003318)<br>
3. Harshvardhan Gupta (2003310) <br>
● Team Code: HaKaPr<br>
● Course Instructor: Dr. Satyanath Bhat<br>
● Course: Machine Learning (CS-331)<br>


#DEEP LEARNING

####1 Implement the following operations (forward and backward pass)<br>
(a) Matrix multiplication layer W X<br>
(b) Bias addition layer<br>
(c) Mean squared loss layer<br>
(d) Soft max layer<br>
(e) Sigmoid layer<br>
(f) Cross entropy loss layer<br>


In [ ]:
import numpy as np

In [ ]:
class MatrixMultiplicationLayer:
    def __init__(self, W):
        self.W = W
        self.X = None
        self.dW = W

    def forward(self, X):
        self.X = X
        return np.dot(X, self.W)  #returns the WX matrix here X=> features and W => Weights 

    def backward(self, dif, learning_rate):
        self.dW = np.dot(self.X.T, dif)
        for i in range(len(self.W)):
          self.W[i] -= learning_rate*self.dW[i]

In [ ]:
import numpy as np

class BiasAdditionLayer:
    def __init__(self, dim):
        self.dim = dim
        self.b = np.ones(dim)
        self.y = None

    def forward(self, y):
        self.y = y
        for yi in y :
          for i in range(self.dim):
            yi[i] += self.b[i]   # adding bias matrix in y
        return y

    def backward(self, dif, learning_rate):
        d_b = 0
        for j in range(self.dim) :
          d_b = 0
          for i in range(len(dif)) :
            d_b += dif[i][j]
          d_b /= len(dif)
          self.b[i] -= learning_rate*d_b


In [ ]:
import numpy as np

class MeanSquaredLossLayer:
    def __init__(self):
        self.y_pred = None
        self.y_true = None

    def forward(self, y_pred, y_true):
        self.y_pred = y_pred
        self.y_true = y_true
        loss = 0
        for i in range(len(self.y_true)) :
           loss += (self.y_pred[i][0]-self.y_true[i])**2   #calculating the squared loss 
        loss /= 2*len(self.y_true)    # taking the mean of squared loss
        return loss  #returning the mean squared loss

    def backward(self):
        dif = self.y_pred 
        for i in range(len(self.y_true)) :  
           dif[i][0] -= self.y_true[i]
        return dif

In [ ]:
import numpy as np
class SoftmaxLayer:
    def __init__(self):
        self.Y_in = None
        self.s_out = None
        
    def forward(self, Y_out):
      # below is the code for softmax activation function:
      # exp(-Pij)/∑j(exp(-Pij))

        mx = np.max(Y_out, axis=1)
        for i in range(len(Y_out)) :
          for j in range(len(Y_out[0])) :
            Y_out[i][j] -= mx[i]
        self.Y_in = Y_out                        
        self.Y_in = np.exp(self.Y_in)            
        self.s_out = self.Y_in / np.sum(self.Y_in, axis=1)
        return self.s_out
        
    
    def backward(self, dout):
        self.ds = np.diag(self.s_out.reshape(-1)) - np.dot(self.s_out.T, self.s_out)
        self.ds = np.dot(dout, self.ds)
        return self.ds


In [ ]:
import numpy as np

class Sigmoid:
    def __init__(self):
        self.params = []

    def forward(self, x):
        # code for sigmoid function
        self.x = x
        return 1 / (1 + np.exp(-x))

    def backward(self, grad_output):
      
      # dout is the gradient of the loss with respect to the output activations
        s = 1 / (1 + np.exp(-self.x))
        return grad_output * s * (1 - s)

In [ ]:

# The cross-entropy loss is defined as:

# L = - ∑ yi * log(pi)

# Where L is the loss, yi is the true label for the ith example and pi is the predicted probability of the true label for the ith example.


import numpy as np

class CrossEntropyLayer:
    def __init__(self):
        self.CE_out = None
        self.epsln = 10**(-10)
        self.Y_true = None
        self.s_out = None

    def forward(self, Y_true, s_out):
        self.Y_true = Y_true
        self.s_out = s_out
        #self.CE_out = -np.sum(Y_true * np.log(s_out+self.epsln) + (Y_true-1)*np.log(1-s_out+self.epsln), axis=0)
        self.CE_out = 0
        #print(len(self.Y_true), "sd", len(self.Y_true[0]), len(self.s_out))

        for i in range(len(self.Y_true[0])) :
          if(Y_true[0][i]==0) :
            self.CE_out += -1*np.log(1-self.s_out[i]+self.epsln)
          else :
            self.CE_out += -1*np.log(self.s_out[i]+self.epsln)
        return self.CE_out

    def backward(self):
        #print(Y_true)
        L_out  = self.Y_true 
        for i in range(len(self.Y_true[0])) :
          if(self.Y_true[0][i]==0) :
            L_out[0][i] = -1/(1-self.s_out[i]+self.epsln)
          else :
            L_out[0][i] = -1/(self.s_out[i]+self.epsln)

        return L_out
       

###2. Using the sklearn.load boston() function, obtain boston house pricing dataset. Train a regression model using the operations implemented above. You need to write a stochastic gradient descent function to train.

In [ ]:
from sklearn.datasets import load_boston
boston = load_boston()

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

X_all = boston.data
X_all = (X_all-np.mean(X_all, axis=0))/np.std(X_all, axis=0)
X_train, X_test, Y_true, Y_test = train_test_split(X_all, boston.target, test_size=0.1)


def sgd(X_train, Y_train, learning_rate):
  size = len(X_train)
  W = np.random.randn(X_train.shape[1], 1)
  multiplication_layer = MatrixMultiplicationLayer(W)
  bias_layer = BiasAdditionLayer(1)
  loss_layer = MeanSquaredLossLayer()
  #learning_rate = 0.00001

  

  for i in range(1000000) :
    index = np.random.randint(low = 0,high = size)
    _x = X_train[index:index+1]
    _y = Y_train[index:index+1]
  # Forward pass:
    y_pred = multiplication_layer.forward(_x)  # calculating the matrix multiplication WX
    y_pred = bias_layer.forward(y_pred)        # adding the bias to get the predicted output layer
    loss = loss_layer.forward(y_pred, _y)      # calculating the loss

    dif = loss_layer.backward()
    bias_layer.backward(dif, learning_rate)    
    multiplication_layer.backward(dif, learning_rate)

  return multiplication_layer.W, bias_layer.b


def testing(X_test, Y_test, W, bias):
  Y_pred = np.dot(X_test, W)
  error = 0
  sum = 0
  for i in range(len(Y_test)) :
    Y_pred[i] += bias
    error += abs(Y_pred[i][0] - Y_test[i])
    sum += Y_test[i]
  error /= sum
  _multiplication_layer = MatrixMultiplicationLayer(W)
  _bias = BiasAdditionLayer(1)
  _bias.b = bias

  
  _loss_layer = MeanSquaredLossLayer()

  # Forward pass:
  _y_pred = _multiplication_layer.forward(X_test)           # calculating the matrix multiplication WX
  _y_pred = _bias.forward(_y_pred)                          # adding the bias to get the predicted output layer          
  meanSqLoss = _loss_layer.forward(_y_pred, Y_test)         # calculating the mean squared loss
  print("Final mean square loss layer on test data: ", meanSqLoss)  
  return error       



W, bias = sgd(X_train, Y_true, 0.00001)
#Wc, biasc = cboost.sgd(X_train, Y_true, 0.00001)
mean_error = testing(X_test, Y_test, W, bias)
#print(mean_error)
print("Average Error: {:.2f}%".format(mean_error * 100))

Final mean square loss layer on test data:  19.233801542202443
Average Error: 15.26%


### #3. Load the iris dataset in sklearn. This data sets consists of 3 different types of irises’ (Setosa, Versicolour, and Virginica) petal and sepal length, stored in a 150x4 numpy.ndarray. Using the operations implemented above create a multi-class classifier (Cross entropy loss + soft max)

In [ ]:
from sklearn.datasets import load_iris  #import load iris dataset from sklearn
import numpy as np
from sklearn.model_selection import train_test_split

iris = load_iris()
X, Y_ = iris.data, iris.target
Y = np.zeros((Y_.shape[0], 3))  #initialsing with zeroes
Y[np.arange(Y_.shape[0]), Y_] = 1

X = (X-np.mean(X, axis=0))/np.std(X, axis=0)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1)



def sgd2(X_train, Y_train, learning_rate) :
  size = len(X_train)

  
  W = np.random.randn(X.shape[1], 3) 
  multiplication_layer = MatrixMultiplicationLayer(W) 
  bias_layer = BiasAdditionLayer(3)   
  soft_max_layer = SoftmaxLayer()    
  entropy_loss_layer = CrossEntropyLayer()   
  
  for i in range(100000) :
    index = np.random.randint(low = 0,high = size)
    _x = X_train[index:index+1]
    _y = Y_train[index]
    # Forward pass:
    y_pred = multiplication_layer.forward(_x)   # calculating the matrix multiplication WX
    y_pred = bias_layer.forward(y_pred)         # adding the bias to get the predicted output layer
    s_out = soft_max_layer.forward(y_pred)      # applying softmax activation 
    loss = entropy_loss_layer.forward(s_out, _y)  # Calculating the Cross entropy Loss
    if(i%200<5) :
      print(loss)
    # Backward pass:
    dif = entropy_loss_layer.backward()
    dw = soft_max_layer.backward(dif)
    bias_layer.backward(dw, learning_rate)
    multiplication_layer.backward(dw, learning_rate)

  return multiplication_layer.W, bias_layer.b 


def testing(X_test, Y_test, W, bias) :
    multiplication_layer = MatrixMultiplicationLayer(W)
    bias_layer = BiasAdditionLayer(3)
    bias_layer.b = bias 
    soft_max_layer = SoftmaxLayer()
    entropy_loss_layer = CrossEntropyLayer()
    loss = 0
    for i in range(len(X_test)) :
      _x = X_test[i:i+1]
      _y = Y_test[i]
      y_pred = multiplication_layer.forward(_x)
      y_pred = bias_layer.forward(y_pred)
      s_out = soft_max_layer.forward(y_pred)
      print(s_out, _y)
      for i in range(len(_y)) :
        print(loss)
        if(abs(s_out[0][i]-_y[i])>0.3) :
          loss += 1
          break
        #loss += abs(s_out[0][i] - _y[i])

    loss /=len(X_test)
    print(len(X_test))
    return loss
    

W, bias = sgd2(X_train, Y_train, 0.0001)
error = testing(X, Y, W, bias)
# print("Error percentage in testing: ", error*100)
#print(mean_error)
print("Error percentage in testing: {:.2f}%".format(error * 100))

46.05170185978092
23.025850929740457
-3.000000248071113e-10
-3.000000248071113e-10
-3.000000248071113e-10
46.05170185978091
-3.000000248071113e-10
46.05170185978092
46.05170185978092
-3.000000248071113e-10
-3.000000248071113e-10
46.05170185978091
-3.000000248071113e-10
46.05170185978092
46.05170185978092
-3.000000248071113e-10
-3.000000248071113e-10
46.05170185978092
-3.000000248071113e-10
46.05170185978091
46.05170185978091
46.05170185978092
-3.000000248071113e-10
46.05170185978092
46.05170185978092
46.05170185978092
-3.000000248071113e-10
46.05170185978092
46.05170185978092
46.05170185978092
46.05170185978092
-3.000000248071113e-10
-3.000000248071113e-10
46.05170185978092
46.05170185978092
46.05170185978092
46.05170185978091
-3.000000248071113e-10
-3.000000248071113e-10
-3.000000248071113e-10
46.05170185978092
46.05170185978092
46.05170185978092
46.05170185978092
-3.000000248071113e-10
46.05170185978092
46.05170185978092
46.05170185978092
46.05170185978092
46.05170185978092
-3.000000